<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/multitask/pre-training/function%20documentation%20generation/php/small_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Predict the documentation for php code using codeTrans multitask training model</h3>**
<h4>You can make free prediction online through this 
<a href="https://huggingface.co/SEBIS/code_trans_t5_small_code_documentation_generation_php_multitask">Link</a></h4> (When using the prediction online, you need to parse and tokenize the code first.)

**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q transformers sentencepiece

     |████████████████████████████████| 1.4MB 5.6MB/s 
     |████████████████████████████████| 1.1MB 19.1MB/s 
     |████████████████████████████████| 890kB 17.4MB/s 
     |████████████████████████████████| 2.9MB 33.3MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

**2. Load the token classification pipeline and load it into the GPU if avilabile**

In [3]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_small_code_documentation_generation_php_multitask"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_small_code_documentation_generation_php_multitask", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


**3 Give the code for summarization, parse and tokenize it**

In [4]:
code = "public static function update ( $ table ) { if ( ! is_array ( $ table ) ) { $ table = json_decode ( $ table , true ) ; } if ( ! SchemaManager :: tableExists ( $ table [ 'oldName' ] ) ) { throw SchemaException :: tableDoesNotExist ( $ table [ 'oldName' ] ) ; } $ updater = new self ( $ table ) ; $ updater -> updateTable ( ) ; }" #@param {type:"raw"}

In [5]:
!pip install tree_sitter
!git clone https://github.com/tree-sitter/tree-sitter-php

     |████████████████████████████████| 112kB 4.2MB/s 
  Created wheel for tree-sitter: filename=tree_sitter-0.2.0-cp36-cp36m-linux_x86_64.whl size=297378 sha256=9575e08f9ff83f91d270a024c3eebd858352c042aa6f10b14e377c6d126eb355
  Stored in directory: /root/.cache/pip/wheels/3b/a6/01/2290cc8441301a07e7717e9e03c6bbc0388f71c6bf1f7f37c1
Successfully built tree-sitter
Cloning into 'tree-sitter-php'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 4459 (delta 252), reused 224 (delta 126), pack-reused 4075
Receiving objects: 100% (4459/4459), 70.14 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (3298/3298), done.


In [6]:
from tree_sitter import Language, Parser

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-php']
)

PHP_LANGUAGE = Language('build/my-languages.so', 'php')
parser = Parser()
parser.set_language(PHP_LANGUAGE)

In [7]:
def get_string_from_code(node, lines):
  line_start = node.start_point[0]
  line_end = node.end_point[0]
  char_start = node.start_point[1]
  char_end = node.end_point[1]
  if line_start != line_end:
    code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))
  else:
    code_list.append(lines[line_start][char_start:char_end])

def my_traverse(node, code_list):
  lines = code.split('\n')
  if node.child_count == 0:
    get_string_from_code(node, lines)
  elif node.type == 'string':
    get_string_from_code(node, lines)
  else:
    for n in node.children:
      my_traverse(n, code_list)
 
  return ' '.join(code_list)

In [8]:
tree = parser.parse(bytes(code, "utf8"))
code_list=[]
tokenized_code = my_traverse(tree.root_node, code_list)
print("Output after tokenization: " + tokenized_code)

Output after tokenization: public static function update ( $ table ) { if ( ! is_array ( $ table ) ) { $ table = json_decode ( $ table , true ) ; } if ( ! SchemaManager :: tableExists ( $ table [ 'oldName' ] ) ) { throw SchemaException :: tableDoesNotExist ( $ table [ 'oldName' ] ) ; } $ updater = new self ( $ table ) ; $ updater -> updateTable ( ) ; }


**4. Make Prediction**

In [9]:
pipeline([tokenized_code])

Your max_length is set to 512, but you input_length is only 98. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': 'Update a table .'}]